# Beginner workflow

This tutorial is a beginner workflow for processing data, visualising the object store and retrieving and visualising data.

### Check installation

This tutorial assumes that you have installed `openghg`. To ensure install has been successful you can open an `ipython` console and import openghg

In a terminal type

```bash
ipython
```

Then import `openghg` and print the version string associated with the version you have installed. If you get something like the below `openghg` is installed correctly.

```ipython
In [1]: import openghg
In [2]: openghg.__version__
Out[2]: '0.0.1'
```

If you get an ``ImportError`` please go back to the install section of the documentation.




### Notebooks

If you haven't used Jupyter notebooks before please see [this introduction](https://realpython.com/jupyter-notebook-introduction/).

## 1. Setting up our environment

First the notebook sets up the environment needed to create the object store at our desired location. By default this location
is at ``/tmp/openghg_store``. For the purposes of this tutorial this path is fine but as it is a temporary directory it may not survive a
reboot of the computer. 

If you want to create an object store that survives a reboot you can change the path to anything you like. We
recommened a path such as ``~/openghg_store`` which will create the object store in your home directory in a directory called ``openghg_store``.

In [1]:
import sys
sys.path.insert(0, "/home/gar/Documents/Devel/RSE/openghg")

from openghg.modules import ObsSurface
from openghg.objectstore import visualise_store
from openghg.localclient import get_obs_surface, RankSources



import glob
from pathlib import Path
import os
import tempfile

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


### Set an environment variable for the OpenGHG object store

Here we create a temporary directory but you can use any folder you like by setting a path in place of `tmp_dir.name`. The object store created by this notebook will only have a lifetime as long as the notebook, if you want to create a longer lived object store set a path below.

In [2]:
tmp_dir = tempfile.TemporaryDirectory()
os.environ["OPENGHG_PATH"] = tmp_dir.name # "/tmp/openghg_store"


## 2. Processing data

First we want to process some files from our local directory

In [3]:
decc_file = "../data/DECC/bsd.picarro.1minute.248m.dat"

We can pass this filepath to `ObsSurface.read_file`. We must also tell it the type of data we want it to process, DECC data is CRDS. We also pass in the site code and the name of the network.

In [4]:
decc_results = ObsSurface.read_file(filepath=decc_file, data_type="CRDS", site="bsd", network="DECC")

  0%|          | 0/1 [00:00<?, ?it/s]

Processing: bsd.picarro.1minute.248m.dat:   0%|          | 0/1 [00:00<?, ?it/s]

Processing: bsd.picarro.1minute.248m.dat: 100%|██████████| 1/1 [00:00<00:00, 11.78it/s]

Here `decc_results` will give us a dictionary with the UUIDs (universally unique identifiers) for each of the Datasources the data has been assigned to. This tells us that the data has been processed and stored correctly.

## A note on Datasources

Datasources are objects that are stored in the object store (add link to object store notes) that hold the data and metadata associated with each measurement we upload to the platform.

For example, if we upload a file that contains readings for three gas species from a single site at a specific inlet height OpenGHG    will assign this data to three different Datasources, one for each species. Metadata such as the site, inlet height, species, network etc are stored alongside the measurements for easy searching. 

Datasources can also handle multiple versions of data from a single site, so if scales or other factors change multiple versions may be stored for easy future comparison.

#### Note

When you run this notebook different UUIDs will be created for the Datasources. This is expected as 
each time a Datasource is created from scratch it is assigned a unique UUID.


In [5]:
decc_results

defaultdict(dict,
            {'processed': {'bsd.picarro.1minute.248m.dat': {'ch4': '553ed001-6b77-469c-b605-643db90daa66',
               'co2': 'a7c87041-1947-4ab8-ae39-ff44a4890029',
               'co': '1999163d-36e9-45f4-bb7e-587fdb9f6f70'}}})

We can now process the AGAGE data. The functions that process the AGAGE data expect data to have an accompanying precisions file. For each data file we create a tuple with the data filename and the precisions filename. A simpler method of uploading these file types is planned.

We must create a `tuple` for each pair

```python
list_of_tuples = [(data_filepath, precision_filepath), (d1, p1), (d2, p2), ...]
```

In [6]:
agage_tuples = [('../data/AGAGE/capegrim-medusa.18.C', '../data/AGAGE/capegrim-medusa.18.precisions.C')]

Then we process the files as we did before the with DECC data, but this time changing the data type to the `GCWERKS` type and the network to `AGAGE`.

In [7]:
agage_results = ObsSurface.read_file(filepath=agage_tuples, data_type="GCWERKS", site="CGO", network="AGAGE")

  0%|          | 0/1 [00:00<?, ?it/s]

Processing: capegrim-medusa.18.C:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/hostedtoolcache/Python/3.7.10/x64/lib/python3.7/site-packages/openghg/modules/_gcwerks.py:288: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[gas_name + " integration_flag"] = (data[column].str[1] != "-").astype(int)
/opt/hostedtoolcache/Python/3.7.10/x64/lib/python3.7/site-packages/openghg/modules/_gcwerks.py:287: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[gas_name + " status_flag"] = (data[column].str[0] != "-").astype(int)
/opt/hostedtoolcache/Python/3.7.10/x64/lib/python3.7/site-packages/openghg/modules/_gcwerks.py:317: PerformanceWarning: DataFrame is highly fragmented.  This

Processing: capegrim-medusa.18.C: 100%|██████████| 1/1 [00:01<00:00,  1.39s/it]

Processing: capegrim-medusa.18.C: 100%|██████████| 1/1 [00:01<00:00,  1.39s/it]

When viewing `agage_results` there will be a large number of Datasource UUIDs shown due to the large number of gases in each data file

In [8]:
agage_results

defaultdict(dict,
            {'processed': {'capegrim-medusa.18.C': {'nf3_70m': '107f3d5b-4283-463e-a40c-924c050a7f8c',
               'cf4_70m': '3ffb5a6e-cfd2-4d33-a467-b7476db5bf73',
               'pfc116_70m': '6ec665f2-5c4f-4b6a-b7ef-74459207cfa2',
               'pfc218_70m': '5fc5efef-9d07-49ea-bbdb-372ee6ec09cb',
               'pfc318_70m': '9e74006f-6c29-4cb1-a4f9-0d5bbc4c0665',
               'c4f10_70m': 'd160befb-72c4-472b-83d3-4950d56a25bb',
               'c6f14_70m': 'ce8d74db-650f-43c2-aa4d-d53bd81427e8',
               'sf6_70m': 'e541eafc-079d-4ca3-97e8-2dc3bf56ba50',
               'so2f2_70m': 'd3514687-3024-445f-a6a5-1efed5bb2e83',
               'sf5cf3_70m': '0bad1fbe-b163-4799-a7c6-07c22f4b114f',
               'hfc23_70m': '6a033cc4-bbf3-47ef-9dcd-c8da310c544d',
               'hfc32_70m': '700215fc-766f-4401-8a94-ee64e1b80a17',
               'hfc125_70m': '0de9ea14-2896-4a24-a43c-f26cce4fa645',
               'hfc134a_70m': '33140b12-d7cc-4a1d-8a96-edfd301

## 3. Visualising the object store

Now that we have a simple object store created we can view the objects within it in a simple force graph model. To do this we use the `view_store` function from the `objectstore` submodule. Note that the cell may take a few moments to load as the force graph is created.

In the force graph the central blue node is the `ObsSurface` node. Associated with this node are all the data processed by it. The next node in the topology are networks, shown in green. In the graph you will see `DECC` and `AGAGE` nodes. From these you'll see site nodes in red and then individual datasources in orange.

#### Note

The object store visualisation created by this function is commented out here and won't be visible in the documentation but can be uncommented and run when you use the notebook version.


In [9]:
# visualise_store()

Now we know we have this data in the object store we can search it and retrieve data from it.

## 4. Retrieving data 

To retrieve data from the object store we can use the `get_obs_surface` function from the `localclient` submodule. This allows us to retrieve and view the data stored.

In [10]:
data = get_obs_surface(site="bsd", species="co", network="AGAGE")

If we view data we expect an `ObsData` object to have been returned

In [11]:
data

ObsData(data=<xarray.Dataset>
Dimensions:                    (time: 105)
Coordinates:
  * time                       (time) datetime64[ns] 2014-01-30T10:52:30 ... ...
Data variables:
    mf                         (time) float64 204.6 200.8 201.5 ... 196.9 196.3
    mf_variability             (time) float64 6.232 5.934 5.176 ... 6.031 6.879
    mf_number_of_observations  (time) float64 26.0 26.0 25.0 ... 26.0 26.0 25.0
Attributes: (12/22)
    data_owner:           Simon O'Doherty
    data_owner_email:     s.odoherty@bristol.ac.uk
    inlet_height_magl:    248m
    comment:              Cavity ring-down measurements. Output from GCWerks
    Conditions of use:    Ensure that you contact the data owner at the outse...
    Source:               In situ measurements of air
    ...                   ...
    sampling_period:      60
    inlet:                248m
    port:                 8
    type:                 air
    network:              decc
    scale:                WMO-X2014A, meta

First we tell `matplotlib` that we are plotting inside a Jupyter notebook, this ensures a plot with controls is created.

In [12]:
%matplotlib notebook

INFO:matplotlib.font_manager:Failed to extract font properties from /usr/share/fonts/truetype/noto/NotoColorEmoji.ttf: In FT2Font: Can not load face.  Unknown file format.


INFO:matplotlib.font_manager:generated new fontManager


In [13]:
example_data = data.data
mol_frac = example_data.mf
mol_frac.plot()

<IPython.core.display.Javascript object>

## 5. Ranking data

The dates that the data from Heathfield retrieved above overlap. If we want to easily retrieve the highest quality data from Heathfield over a range of dates we don't want to have to repeatedly check which was the correct inlet/instrument for a given daterange. This problem is solved using ranking. 

A given inlet on a specific instrument at a site can be given a rank for a daterange. To do this we use the `RankSources` class from the `localclient` submodule.

# The rest of tutorial requires updating, the cells below will not work.

In [14]:
#r = RankSources()

#r.get_sources(site="hfd", species="co")

The returned dictionary gives us two keys, one for each inlet height. To rank a source we use the `set_rank` method which expects two arguments: `rank_key` which is the key given to each source in the `dict` above and `rank_data` a dictionary of the form

```python
rank_data = {co2_hfd_50m_picarro: {1: [daterange_1], 2: [daterange_2]}}
```

We can create this dictionary using a helper method of `RankSources` called `create_daterange` as shown below.

In [15]:
#daterange_100m = r.create_daterange(start="2013-11-01", end="2016-01-01")   

This creates a daterange string that will be understood by `openghg`. We can then place this in a list to create our `rank_data` dictionary.

In [16]:
#rank_data = {"co_hfd_100m_picarro": {"1": [daterange_100m]}}

In [17]:
#rank_data

Now we can set the rank of the network using `set_rank`

In [18]:
#r.set_rank(rank_key="co_hfd_100m_picarro", rank_data=rank_data)

We can now check the rank for this inlet again to check it's been set correctly

In [19]:
#r.get_sources(site="hfd", species="co")

We can now see

```python
'co_hfd_100m_picarro': {'rank': defaultdict(list, {'1': ['2013-11-01T00:00:00_2016-01-01T00:00:00']})
```

Which tells us the rank was set correctly over the daterange that we specified. We can now search for data and we'll automatically get the highest ranked data.

Let's search for CO2 data at Heathfield between 2014 - 2015, dates covered by both inlets.

In [20]:
#updated_data = get_obs_surface(site="hfd", species="co", network="AGAGE")

In [21]:
#updated_data

Now we get the highest ranked data returned to us without the need to specify an inlet height or instrument.

If we know that we want data from the 50m inlet we can still specify this in the search and get that data

In [22]:
#fiftym_data = get_obs_surface(site="hfd", species="co", network="AGAGE", inlet="50m")

In [23]:
#fiftym_data

## 6. Viewing ranked data

We can also view the ranks we have given to data with a similar layout to the object store visualisation we created earlier.

To do this we use the `visualise_rankings` method of of the `RankSources` class. In this figure we'll only see Datasources that contain ranked data. Hover over the nodes for further information.

#### Note

The rankings visualisation created by this function is commented out here and won't be visible in the documentation but can be uncommented and run when you use the notebook version.


In [24]:
# r.visualise_rankings()

## 7. Cleanup

If you used the `tmp_dir` as a location for your object store at the start of the tutorial you can run the cell below to remove any files that were created.

In [25]:
tmp_dir.cleanup()

## 7. What's next?

Further tutorials will be added soon. If you want to explore the internal workings of OpenGHG please checkout the Developer API documentation, if you would like contribute to the project we welcome pull requests to both the code and the documentation. For help and guidance on contributing check our contributing page.